In [1]:
gROOT->ProcessLine(".L common/gallery_includes.h+O");
gROOT->ProcessLine(".L common/larsoftobj_includes.h+O");
gROOT->ProcessLine(".L common/My_ROOT_Utilities.h");

In [2]:
//for space charge
gROOT->ProcessLine(".L $GSL_LIB/libgsl.so");
gROOT->ProcessLine(".L $UBOONECODE_LIB/libuboone_SpaceCharge.so");
gROOT->ProcessLine(".L common/Utils_SpaceCharge.h+O")

(long) 0


In [3]:
//setup our input file and input tag
std::vector<std::string> filenames;
std::string file_path;
TFile *f_output;

TNtuple *nt;
TCanvas canvas("canvas","MyCanvas");

art::InputTag vtx_tag;
art::InputTag pfp_tag;
art::InputTag gen_tag;

//geometry
float X_LENGTH=256.0;
float Y_HALF_HEIGHT=116.5;
float Z_LENGTH=1037.0;

float PROTON_THRESHOLD=0.030;

std::unique_ptr<spacecharge::SpaceChargeMicroBooNE> sce;
CreateSCE(sce);
geo::Vector_t offsets;

In [4]:
%%cpp -d
void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const&,
                    int ntab=0);

void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const& pfp_vec,
                    int ntab)
{
    for(size_t i_d=0; i_d<pfp_vec[i_pfp].NumDaughters(); ++i_d){
        for(int it=0; it<ntab; ++it) std::cout << "\t";
        std::cout << "Found daughter particle id=" << pfp_vec[i_pfp].Daughters()[i_d] 
                  << ", code=" << pfp_vec[pfp_vec[i_pfp].Daughters()[i_d]].PdgCode() << std::endl;
        PrintDaughters(pfp_vec[i_pfp].Daughters()[i_d],pfp_vec,ntab+1);
    }
    return;
}

In [5]:
bool verbose=false;

filepath = "/pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/";
filenames = GetFileNameList("/uboone/data/users/wketchum/nue_tune3_reco2_filelist.txt",filepath);

f_output = new TFile("/uboone/data/users/wketchum/nue_tune3_vtx_ntuple_tmp.root","RECREATE");

nt = new TNtuple("nt","MyNtuple","nu_x:nu_y:nu_z:nu_e:q2:lep_p:n_p:p_ke:vtx_x:vtx_y:vtx_z:vtx_dist:pfp_primary:pfp_pdg");

vtx_tag = { "pandoraNu" };
pfp_tag = { "pandoraNu" };
gen_tag = { "generator" };


In [6]:
//filenames.size()
//for(auto & fname : filenames)
    //std::cout << fname << std::endl;
filenames.resize(300);

In [7]:
for(gallery::Event ev(filenames) ; !ev.atEnd(); ev.next()) {
    if(verbose)
        std::cout << "Processing " 
                  << "Run " << ev.eventAuxiliary().run() << ", "
                  << "Event " << ev.eventAuxiliary().event() << endl;
    
    auto const& mctruth_handle = ev.getValidHandle<std::vector<simb::MCTruth>>(gen_tag);
    auto const& truth_vec(*mctruth_handle);

    if(truth_vec.size()==0){
        std::cout << "ERROR! truth_vec size is " << truth_vec.size() << std::endl;
        break;
    }
       
    auto const& nu = truth_vec[0].GetNeutrino();
    geo::Point_t nu_position(nu.Nu().Vx(),nu.Nu().Vy(),nu.Nu().Vz());
    nu_position = nu_position + sce->GetPosOffsets(nu_position);
    if(nu_position.X()<0.0+10.0 || nu_position.X()>X_LENGTH-10.0 ||
       nu_position.Y()<-1*Y_HALF_HEIGHT+20.0 || nu_position.Y()>Y_HALF_HEIGHT-20.0 ||
       nu_position.Z()<0.0 || nu_position.Z()>Z_LENGTH-10.0 )
        continue;

    if(verbose)
        std::cout << "Neutrino position is (" 
                  << nu.Nu().Vx() << ","
                  << nu.Nu().Vy() << ","
                  << nu.Nu().Vz() << ")"
                  << std::endl;

    float lep_energy = -999, lep_p = -999;
    if(nu.Mode()==0) { //CC
        lep_energy = nu.Lepton().E();
        lep_p = nu.Lepton().P();
    }
    int n_proton=0; float max_proton_ke=0.0;

    for(size_t i_p=0; i_p<truth_vec[0].NParticles(); ++i_p){
        auto const& p = truth_vec[0].GetParticle(i_p);

        if(p.StatusCode()!=1 || p.PdgCode()!=2212) continue;
        
        auto ke = (p.E()-p.Mass());
        if(ke > PROTON_THRESHOLD) ++n_proton;
        if(ke>max_proton_ke) max_proton_ke = ke;        
    }

    auto const& pfp_handle = ev.getValidHandle<std::vector<recob::PFParticle>>(pfp_tag);
    auto const& pfp_vec(*pfp_handle);
    if(verbose)
        std::cout << "There are " << pfp_vec.size() << " PFParticles in this event." << std::endl;

    for (auto const& pfp : pfp_vec){
        if(!pfp.IsPrimary()) continue;
        if(verbose){
            std::cout << "Found primary particle id=" << pfp.Self() << ", code=" << pfp.PdgCode() << std::endl;
            PrintDaughters(pfp.Self(),pfp_vec,1);
        }
    }
    
    
    auto const& vtx_handle = ev.getValidHandle<std::vector<recob::Vertex>>(vtx_tag);
    auto const& vtx_vec(*vtx_handle);
    if(verbose)
        std::cout << "There are " << vtx_vec.size() << " vertices in this event." << std::endl;

    art::FindOneP<recob::Vertex> vtx_per_pfp(pfp_handle,ev,pfp_tag);
    art::Ptr<recob::Vertex> vtxptr;
    double min_dist=9e9; int i_pfp_nu=-1; geo::Point_t pfp_position;
    
    for(size_t i_pfp=0; i_pfp<pfp_vec.size(); ++i_pfp)
    {
        
        vtx_per_pfp.get(i_pfp,vtxptr);
        if(vtxptr.isNonnull()){
            auto const& vtx(*vtxptr);
            if(verbose)
                std::cout << "For pfparticle " << i_pfp << " found vertex at "
                          << "(" << vtx.position().X() << ","
                          << ""  << vtx.position().Y() << ","
                          << ""  << vtx.position().Z() << ")"
                          << std::endl;
            
            if(sqrt((vtx.position()-nu_position).Mag2()) < min_dist){
                min_dist = sqrt((vtx.position()-nu_position).Mag2());
                i_pfp_nu = i_pfp;
                pfp_position = vtx.position();
            }
        }
        
            
    }
    
    nt->Fill(nu_position.X(),nu_position.Y(),nu_position.Z(),
             nu.Nu().E(),nu.QSqr(),lep_p,n_proton,max_proton_ke,
             pfp_position.X(),pfp_position.Y(),pfp_position.Z(),
             min_dist,
             pfp_vec[i_pfp_nu].IsPrimary(),pfp_vec[i_pfp_nu].PdgCode());
    
    
}//end loop over events


Warning in <TClass::Init>: no dictionary for class art::DoNotRecordParents is available


Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_3634_20180207T203914_gen2_71bc0c41-9b83-4568-908d-ec73e29e88bf_20180211T025322_reco1_20180211T042228_reco2.root


Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoConstants.h
   requested to autoload type larlite::geo::View_t
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoConstants.h
   requested to autoload type larlite::geo::SigType_t
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::TPCID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::PlaneID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::WireID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for larlite_base.h
   requested to autoload type larlite::Message
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntr

Closing file, read 2917051 bytes in 734 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_1899_20180207T202308_gen2_5edecb5f-187c-4a2d-bb3d-c632d040a763_20180211T185730_reco1_20180211T195407_reco2.root
Closing file, read 2921764 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_549_20180207T201409_gen2_f2ff74d2-fac3-4959-9482-34c8b015ae7f_20180211T025046_reco1_20180211T041029_reco2.root
Closing file, read 2914490 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_518_20180207T210248_gen2_0905dd68-9bec-414e-99dd-9afce38b8fb3_20180211T0214

Closing file, read 2925936 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_939_20180207T180946_gen2_21b12bf6-f618-4ad3-95ba-dc95d837b943_20180211T174853_reco1_20180211T185730_reco2.root
Closing file, read 2925128 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_229_20180207T193806_gen2_af8b9d13-6899-4deb-b861-3ee943ffcd53_20180211T154634_reco1_20180211T164752_reco2.root
Closing file, read 2925772 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_1274_20180207T174147_gen2_270fa07c-2d62-452f-9a0f-87a8f2e740ea_20180211T1809

...

Closing file, read 2923370 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_4413_20180208T004853_gen2_df259fb8-7f8f-4277-8a36-76f46af19253_20180211T153051_reco1_20180211T171821_reco2.root
Closing file, read 2926377 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_2762_20180208T023016_gen2_8e995f4f-6bdb-47e5-ba2f-86e0f119e121_20180211T170841_reco1_20180211T182630_reco2.root
Closing file, read 2915499 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_3256_20180207T214714_gen2_a87cf33d-cacf-43c9-baf3-b51eb633cd83_20180211T02

Closing file, read 2924458 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_3237_20180207T213248_gen2_1ef5e0f4-4978-4aa7-a9a9-b51c5f25750c_20180211T030512_reco1_20180211T035331_reco2.root
Closing file, read 2921798 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_4572_20180208T034941_gen2_b91b29ed-4f75-482a-a460-44b9e5761c4a_20180212T071501_reco1_20180212T083545_reco2.root
Closing file, read 2922787 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_1864_20180207T231754_gen2_346bc71a-e897-49a4-9a0f-a10233107c5d_20180211T02

Closing file, read 2919333 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_481_20180207T212409_gen2_adc0c412-d5b4-4fc9-8eb6-b1ced228fd37_20180211T193129_reco1_20180211T211236_reco2.root
Closing file, read 2921652 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_3733_20180208T004914_gen2_5477aa55-49eb-48d8-b111-ef6c137effb5_20180211T193047_reco1_20180211T204344_reco2.root
Closing file, read 2915487 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_3852_20180207T220659_gen2_117ccbde-646d-4387-9a8b-614a32621d43_20180211T034

Closing file, read 2913961 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_50_20180208T081711_gen2_aa25c80e-e53a-4b4c-83b2-372952957bec_20180211T185352_reco1_20180211T200056_reco2.root
Closing file, read 2923292 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_3551_20180208T003504_gen2_5434b2c1-7517-44f7-87ab-090fc0db379e_20180211T030455_reco1_20180211T044250_reco2.root
Closing file, read 2917750 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_4974_20180208T040916_gen2_2aef57d2-e3f3-4e5a-b5fb-60dd8769491e_20180211T1533

Closing file, read 2927222 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_259_20180207T192025_gen2_fd8d00d2-938c-45dd-94bd-71361b397ffe_20180211T151116_reco1_20180211T164305_reco2.root
Closing file, read 2919787 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_4024_20180207T202408_gen2_ecc72cdf-f286-4eed-b94c-88f9f4225e77_20180211T030608_reco1_20180211T040441_reco2.root
Closing file, read 2926029 bytes in 409 transactions
Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_761_20180207T175055_gen2_0b8721ab-3599-4bf3-becb-5a5e2d5e9ba9_20180211T0252

In [8]:
f_output->cd();
nt->Write();
f_output->Close();

Todos:

---Space charge correction.

---Understand the fraction of charge in the reconstructed heirarchies relative to the truth deposited. Compare across "well-reconstructed" (close to vertex) and "poorly-reconstructed" (not close to vertex)

---Should look at distance of reconstructed primary to the truth, and not restrict to closest vertex (could be a daughter is super close to the primary too)

---Is there an energy dependence?

---Is there a topology/proton/lepton energy dependence?